In [1]:
from concurrent.futures import ThreadPoolExecutor
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import mysql.connector
import logging
import time

In [2]:
def save_to_excel(df, save_path):
    # 먼저 코퍼스 리스트를 데이터 프레임으로 만들고 저장한다.
    writer = pd.ExcelWriter(save_path)
    df.to_excel(writer, 'Sheet1', index=False, header=True)
    writer.save()
    
# 사용자가 평가한 영화 url 수집
def get_movie_link(url):
    req = requests.get(url)
    content = req.text
    soup = BeautifulSoup(content, 'html.parser')
    movie_url = soup.select('a[href]')
    movie_urls = []

    for link in movie_url:
        if re.search(r'st=mcode&sword' and r'&target=after$', link['href']):
            target_url = 'http://movie.naver.com/movie/point/af/list.nhn' + str(link['href'])
            movie_urls.append(target_url)
    return movie_urls


# 영화 장르 수집
def get_detailed_movie_info(url):
    movie_urls = get_movie_link(url)
    movie_detailed = []
    specific_movie_detailed = []
    movie_title_list = []
    
    useless = ["", "", "", "", "", ""]

    for movie_url in movie_urls:
        req = requests.get(movie_url)
        content = req.text
        soup = BeautifulSoup(content, 'html.parser')
        # movie_title = soup.select("#old_content > div.choice_movie_box > div.choice_movie_info > h5 > a")
        movie_info = soup.select('#old_content > div.choice_movie_box > div.choice_movie_info > table > tr > td')
        
        
#         for m in movie_title:
#             m = re.sub('<.*?>', " ", str(m))
#             m = re.sub('(\s){2,}', "", m)
#             movie_title_list.append(m)
        
        for m_info in movie_info:
            m_info = re.sub('<.*?>', " ", str(m_info))
            m_info = re.sub('(\s){2,}', "", m_info)
            # print("splited", m_info)
            if m_info != '':
                movie_detailed.append(m_info)
                # print("m", m_info)
    
    buff_list = []
    buff_list2 = []
    
    cnt = 0
    for buff in movie_detailed:
        buff_list.append(buff)
        cnt += 1
        if(cnt == 4):
            cnt = 0
            buff_list2.append(buff_list)
            buff_list = []
    movie_detailed = buff_list2
    # print("movie_detailed",movie_detailed)
    
    t_cnt = 0
    m_cnt = 0
    for i in movie_detailed:
        b = []
        for ii in i[0].split("|"):
            b.append(ii)
        b.append(i[1])
        b.append(i[2])
        b.append(i[3].split("/")[0])
        if(len(b) == 6):
            specific_movie_detailed.append(b)
        else:
            specific_movie_detailed.append(useless)
            pass
    
    movie_info_df = pd.DataFrame(specific_movie_detailed[0:],columns=['genre','running_time','release date','director','actor/actress','movie_rating'])
    return movie_info_df


# 사용자의 페이지 링크를 수집한다.
def get_user_page(url):
    req = requests.get(url)
    content = req.text
    soup = BeautifulSoup(content, 'html.parser')

    # a 태그이면서 href 속성을 갖는 경우 탐색
    user_page = soup.select('a[href]')
    user_page_urls = []

    for link in user_page:
        if re.search(r'st=nickname&sword' and r'page', link['href']):
            target_url = 'http://movie.naver.com' + str(link['href'])
            user_page_urls.append(target_url)
            # print('1. target_url: ', target_url)
            
    # print('2. user_page_urls: ', user_page_urls)
        # '다음' 때문에 한페이지가 더 추가되어 처리
    if len(user_page_urls) != 1:
        pop_number = len(user_page_urls) - 1
        # print('3. pop_number', pop_number)
        user_page_urls.pop(pop_number)
    # print(user_page_urls)
    return user_page_urls

In [3]:
def do_crawl(url):
    num = 0
    new_df = pd.DataFrame()
    urls = get_user_page(url)
    # 평점을 10개 이하로 준 유저는 제외한다
    
    if len(urls) >= 2:
        for user_url in urls:
            req = requests.get(user_url)
            # print(req)
            content = req.text
            # print(content)

            # print(user_url, "에 대한 크롤링 시작!")
            soup = BeautifulSoup(content, 'html.parser')

            detailed_movie_info = get_detailed_movie_info(user_url)

            user_post_id = soup.find_all('td', class_='ac num')
            user_id = soup.find_all('a', class_='author')
            title = soup.find_all('td', class_='title')  # a 태그
            review = soup.find_all('td', class_='title')
            # print(review)
            score = soup.find_all('td', class_='point')
            # print(score)
            postDate = soup.find_all('td', class_='num')  # a 태그

            user_post_id_list = []
            for u_p_id in user_post_id:
                # print(u_p_id.get_text())
                user_post_id_list.append(u_p_id.get_text())

            user_id_list = []
            for u_id in user_id:
                # print(u_id.get_text())
                user_id_list.append(u_id.get_text())

            title_list = []
            for t in title:
                # print(t.a.get_text())
                title_list.append(t.a.get_text())

            review_list = []
            for r in review:
                # print("before ",r)
                r = re.sub('<td class=(.*?)>|</td>', " ", str(r))
                r = re.sub('<br>|<br/>', " ", r)
                r = re.sub('<.*?>(.*?)<.*?>', " ", r)
                r = re.sub('(\s){2,}', "", r)
                # print(r)
                review_list.append(r)

            score_list = []
            for s in score:
                s = re.sub('<td class=(.*?)>|</td>', " ", str(s))
                # print(s)
                score_list.append(s)

            postDate_list = []
            for p in postDate:
                # print("before: ", p)
                p = re.sub('<td class="ac num">(.*?)<.*?>'," ", str(p))
                p = re.sub('<td class=(.*?)>|</td>|<br/>', " ", p)
                p = re.sub('<.*?>(.*?)<.*?>', " ", p)
                p = re.sub('(\s){2,}', "", p)
                postDate_list.append(p)

            postDate_list = [d.replace(" ", "") for d in postDate_list if d != " "]
            # print(postDate_list)

            df = pd.DataFrame()
            df['user_post_id'] = user_post_id_list
            df['user_id'] = user_id_list
            df['movie'] = title_list
            df['review'] = review_list
            df['user movie raitng'] = score_list
            df['date of posting'] = postDate_list

            df = pd.concat([df, detailed_movie_info], axis = 1, ignore_index=True)
            print(user_url, " 크롤링 완료")
            user_df = pd.DataFrame(df)
            # user_df.append(df, ignore_index=True)
            num += 1
            # print(num, user_df)
            new_df = pd.concat([new_df, user_df], axis = 0, ignore_index=True)
        # print("---- 전체", url, " 크롤링 완료 ---- ")
        # print(url, "에 대한 전체 결과: ", all_df)
        # print(all_df)  

    return new_df

In [4]:
# start > end 여야 한다.
def getData(start, end):

    total_crawling = []
    
    # 네이버 영화 평점 페이지에 등록된 번호
    # 사용자에 따라 수정 가능 함
    page = start
    if page != 14180475:
        while page > end - 1:
            page = str(page)
            #{}안에 페이지 값이 삽입
            url = "http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword={}&target=after".format(page)

            # 크롤링 함수를 불러 객체로 저장한다.
            result = do_crawl(url)
            total_crawling.append(result)
            # print(result)

            page = int(page)
            page -= 1

        result_crawling = pd.concat(total_crawling, ignore_index=True)
    
    else:
        pass
    
    return result_crawling

In [ ]:
crawling_page_list = [[14180780, 14180400], [14180400, 14180000], [14180000, 14179000]]
num = 0
for pages in crawling_page_list:
    num += 1
    save_path = "D:\\naver_movie_{}_second_version.xlsx".format(num)
    save_to_excel(getData(pages[0], pages[1]), save_path)
    print("진짜 저장? 저장 위치는: ", save_path)

D:\naver_movie_1_second_version.xlsx
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180780&target=after&page=1  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180780&target=after&page=2  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180780&target=after&page=3  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180780&target=after&page=4  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180780&target=after&page=5  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180780&target=after&page=6  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180780&target=after&page=7  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180780&target=after&page=8  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180780&target=after&page=9  크롤링 완료
http://movie.naver.com/movie/point/af/list.nh

http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180711&target=after&page=4  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180711&target=after&page=5  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180711&target=after&page=6  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180709&target=after&page=1  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180709&target=after&page=2  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180708&target=after&page=1  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180708&target=after&page=2  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180708&target=after&page=3  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180708&target=after&page=4  크롤링 완료
http://movie.naver.com/movie/point/af/list.nhn?st=nickname&sword=14180708&target=a

In [ ]:
# 병렬 처리 실패쓰..
#     with ThreadPoolExecutor(max_workers=3) as executor:
#         try:
#             future = executor.submit()
#             # movie_df = pd.concat([movie_df, executor.submit(do_crawl(url))],axis = 0)
#             print('result:', future)
            
#             res = future.result()
#             print('res', res)
            
#             if res != None:
#                 total_crawling.append(res)
    
#         except Exception as exc:
#             print('%r generated an exception: %s' % (url, exc))